In [ ]:
test_df['NumOfPOS'] = None
test_df['NumOfNEG'] = None
test_df['POS_token'] = None
test_df['NEG_token'] = None
test_df['Label'] = None


index_count = 0
for sentence in test_df['Sentence']:
    tokens = word_tokenize(sentence)
    pos_count = 0
    neg_count = 0
    pos_token_list = []
    neg_token_list = []
    
    for token in tokens:
        if token in pos_list:
            pos_count += 1
            pos_token_list.append(token)
        elif token in neg_list:
            neg_count += 1
            neg_token_list.append(token)
        else:
            pass
        
    test_df['NumOfPOS'][index_count] = pos_count
    test_df['POS_token'][index_count] = pos_token_list    # 같은 token이 있어도 중복시킨다
    test_df['NumOfNEG'][index_count] = neg_count
    test_df['NEG_token'][index_count] = neg_token_list
    
    if pos_count > neg_count:
        test_df['Label'][index_count] = 'POS'
    elif pos_count < neg_count:
        test_df['Label'][index_count] = 'NEG'
    else:
        pass
    index_count += 1
    if index_count % 1000 == 0:
        print(index_count)

### Map 함수

In [64]:
df

,Sentence,Tokens,Token,WHY
0,i am a boy,"[i, am, a, boy]","[i, am, a, boy]",NaN
1,i am a girl,"[i, am, a, girl]","[i, am, a, girl]",1.0
2,he is good man,"[he, is, good, man]","[he, is, good, man]",NaN


In [74]:
import nltk
from nltk import word_tokenize

pos_list = ['boy']
neg_list = ['i']
neg_count = 0
pos_token_list = []
neg_token_list = []

def tokenize(x):
    return word_tokenize(x)

def Pos_and_Neg(x):
    pos_count = 0
    neg_count = 0
    a = None
    
    for i in x:
        if i in pos_list:
            pos_count += 1
        if i in neg_list:
            neg_count += 1
            
    if pos_count > neg_count:
        a = 'POS'
    elif neg_count > pos_count:
        a = 'NEG'
    else:
        a = None
            
    return a

def PosNegCount(x):
    pos_neg_ls = []
    pos_count = 0
    neg_count = 0
    
    for i in x:
        if i in pos_list:
            pos_count += 1
        if i in neg_list:
            neg_count += 1
            
    pos_neg_ls.append(pos_count)
    pos_neg_ls.append(neg_count)
    return pos_neg_ls

df['Token'] = df['Sentence'].map(lambda x : tokenize(x))
df['Label'] = df['Token'].map(lambda x: Pos_and_Neg(x))
df['POS_NEG_Count'] = df['Token'].map(lambda x: PosNegCount(x))


In [75]:
df

,Sentence,Tokens,Token,WHY,Label,POS_NEG_Count
0,i am a boy,"[i, am, a, boy]","[i, am, a, boy]",None,None,"[1, 1]"
1,i am a girl,"[i, am, a, girl]","[i, am, a, girl]",NEG,NEG,"[0, 1]"
2,he is good man,"[he, is, good, man]","[he, is, good, man]",None,None,"[0, 0]"


In [88]:
def test(x):
    word = None
    if x > 3:
        word = 'POS'
    if x < 5:
        word = 'Neg'
    return word

df['label'] = df['Token'].apply(lambda x: test(len(x)))

In [89]:
df

,Sentence,Tokens,Token,WHY,Label,POS_NEG_Count,label
0,i am a boy,"[i, am, a, boy]","[i, am, a, boy]",None,None,"[1, 1]",Neg
1,i am a girl,"[i, am, a, girl]","[i, am, a, girl]",NEG,NEG,"[0, 1]",Neg
2,he is good man,"[he, is, good, man]","[he, is, good, man]",None,None,"[0, 0]",Neg


## 선민형님 코드 구동 도와드리기

In [91]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from pprint import pprint
import pymysql
import pandas as pd

raw = pd.read_csv('to승지님께.csv', encoding='utf-8')
db = pymysql.connect(
    host='ec2-13-209-223-88.ap-northeast-2.compute.amazonaws.com',  # DATABASE_HOST
    port=3306,
    user='nlpuser',  # DATABASE_USERNAME
    passwd='1234',  # DATABASE_PASSWORD
    db='nlp',  # DATABASE_NAME
    charset='utf8'
)

SQL = "SELECT * FROM TotalDic"
MPQA = pd.read_sql(SQL, db)

MPQA = MPQA[MPQA['DicType2'] == 'MPQA']
MPQA_list = MPQA[['Rule','Label']]
POS_list = MPQA_list[MPQA_list['Label']=='POS']
pos_list = POS_list['Rule']
pos_list = list(pos_list)
NEG_list = MPQA_list[MPQA_list['Label']=='NEG']
neg_list = NEG_list['Rule']
neg_list = list(neg_list)

df_test = raw.copy()
# pos_list = ['boy']
# neg_list = ['i']
# neg_count = 0

pos_token_list = []
neg_token_list = []

def tokenize(x):
    return word_tokenize(x)

def Pos_and_Neg(x):
    pos_count = 0
    neg_count = 0
    a = None
    for i in x:
        if i in pos_list:
            pos_count += 1
        if i in neg_list:
            neg_count += 1
        if pos_count > neg_count:
            a = 'POS'
        elif neg_count > pos_count:
            a = 'NEG'
        else:
            a = None
    return a

def PosNegCount(x):
    pos_neg_ls = []
    pos_count = 0
    neg_count = 0
    for i in x:
        if i in pos_list:
            pos_count += 1
        if i in neg_list:
            neg_count += 1
    pos_neg_ls.append(pos_count) 
    pos_neg_ls.append(neg_count)
    return pos_neg_ls

df_test['Token'] = df_test['Sentence'].map(lambda x : tokenize(x))
print('Token화 완료')
df_test['POS_NEG_Count'] = df_test['Token'].map(lambda x: PosNegCount(x))
print('POS, NEG 붙이기')
# df_test['Label'] = df_test['Token'].map(lambda x: Pos_and_Neg(x))

df_test

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Token화 완료


KeyboardInterrupt: 